### Global Imports

In [5]:
import urllib.request
from bs4 import BeautifulSoup
import pandas
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
import os
import time

### Global Variables

In [8]:
#extensionPath = '.\Hola-Free-VPN-Proxy-Unblocker-Chrome Web Store_v1.159.227.crx'
executable_path = "chromedriver.exe"
inputFilePath = "D:\\DSIL\\county_case_input.csv"
outputFilePath = "D:\\DSIL\\oscn_details.csv"

### Setting chrome option (if required)

In [ ]:
#os.environ["webdriver.chrome.driver"] = executable_path
#chrome_options = Options()
#chrome_options.add_extension(extensionPath)

### Creating Driver

In [78]:
driver = webdriver.Chrome(executable_path=executable_path)

In [ ]:
driver.get(oscn_url)
details = {}

### Build OSCN Url

In [181]:
def build_oscn_url(countyName,caseNumber):
    oscnBaseUrl = "https://www.oscn.net/dockets/GetCaseInformation.aspx"
    parsed = re.match("([^-]*)-([0-9]*)-([^-]*)(.*)$",caseNumber)
    caseType = parsed.groups()[0]
    caseYear = parsed.groups()[1]
    caseNum = parsed.groups()[2]
    extra = parsed.groups()[3]

    cn = caseType + '-' + caseYear + '-' + caseNum

    s = oscnBaseUrl + "?db=" + countyName.lower() + "&number=" + cn;
    return s
 

### Getting Attorney Information

In [162]:
def get_attorney_info(driver):
    attorneyDetails = {}
    attorneyTable = driver.find_elements_by_css_selector("table[id='TABLE_2'] tr")
    for elem in attorneyTable[1:]:
        attorneyList = elem.text.split("\n")[:-1]
        attorneyDetails["attorney" + "_" + str(attorneyTable.index(elem))] = " ".join(attorneyList)
        attorneyDetails["representedParties" + "_" + str(attorneyTable.index(elem))] = elem.text.split("\n")[-1]
    return attorneyDetails

### Getting Events Information

In [163]:
def get_events_info(driver):
    eventsDetails = {}
    eventsTable = driver.find_elements_by_css_selector("table[id='TABLE_3'] tr")
    for elem in eventsTable[1:]:
        infoTd = elem.find_elements_by_css_selector("td") 
        eventsDetails["event" + "_" + str(eventsTable.index(elem))] = infoTd[0].text
        eventsDetails["party" + "_" + str(eventsTable.index(elem))] = infoTd[1].text
        eventsDetails["docket" + "_" + str(eventsTable.index(elem))] = infoTd[2].text
        eventsDetails["reporter" + "_" + str(eventsTable.index(elem))] = infoTd[3].text
    return eventsDetails

### Getting Docket Information

In [237]:
def get_docket_info(driver):
    docketDetails = {}
    docketTable = driver.find_elements_by_css_selector("table[class='docketlist'] tr")
    for elem in docketTable[1:]:
        if len(elem.text.split("\n")[0].strip()) ==  10:
            docketDetails["docket_date" + "_" + str(docketTable.index(elem))] = elem.text.split("\n")[0].split("  ")[0]
            if len(elem.text.split("\n")[0].split("  ")) > 1:
                docketDetails["docket_code" + "_" + str(docketTable.index(elem))] = elem.text.split("\n")[0].split("  ")[1]
            else:
                docketDetails["docket_code" + "_" + str(docketTable.index(elem))] = ""
    return docketDetails

### Main Flow

In [248]:
inputFile = pandas.read_csv(inputFilePath)
oscnDetailsList = []
for row in inputFile[:10].itertuples():
    oscnUrl = build_oscn_url(row[2],row[4])
    driver.get(oscnUrl)
    
    baseInfo = {"countyName":row[2],"caseNum":row[4],"oscnUrl":oscnUrl}
    attorneyInfo = get_attorney_info(driver)
    eventsInfo = get_events_info(driver)
    docketInfo = get_docket_info(driver)
    
    details =  {**baseInfo,**attorneyInfo, **eventsInfo, **docketInfo}
    oscnDetailsList.append(details)



## Writing Out

In [249]:
output = pandas.DataFrame(oscnDetailsList)
output.to_csv(outputFilePath,index=False)

## Spydering Flow

In [39]:
## Global Variables 
countyName = "oklahoma" # we have make this as list and iterate
startDate = "2016/12/31"
endDate = "2018/01/01"
baseUrl = "http://www.oscn.net/dockets/Results.aspx?"


## Constructing Spyder Url
def construct_spyder_url(county,start,end):
    spyderUrl = baseUrl + "db=" + county + "&dcct=2" + "&FiledDateL=" + start + "&FiledDateH=" + end
    return spyderUrl

## Getting case urls for given Spyder Url
def get_case_urls(countyName,startDate,endDate,driver):
    spyderUrl = construct_spyder_url(countyName,startDate,endDate)
    driver.get(spyderUrl)
    caseList = []
    cases = driver.find_elements_by_css_selector("table[class='caseCourtTable'] tr[class*='resultTableRow']")
    for case in cases:
        caseList.append(case.find_element_by_tag_name("a").get_attribute("href"))
    return caseList


## CaseList mainflow
driver = webdriver.Chrome(executable_path=executable_path)
caseList = get_case_urls(countyName,startDate,endDate,driver)

In [41]:
len(caseList)

500